In [2]:
%display latex

In [3]:
import sys

sys.path.append("..")
from calibration.projector.projector import Projector

In [4]:
lambda_1, lambda_2, lambda_3 = var("lambda_1 lambda_2 lambda_3", domain="real")


def psi(r, n=2):
    assert 1 <= n <= 3
    ret = 1 + lambda_1 * r ^ 2
    if n >= 2:
        ret += lambda_2 * r ^ 4
    if n == 3:
        ret += lambda_3 * r ^ 6
    return ret


psi(var("r"))

lambda_2*r^4 + lambda_1*r^2 + 1

In [5]:
IM = matrix(Projector().camera.intrinsic_matrix)
IM

[1166.6666666666667                0.0              600.0]
[               0.0 1166.6666666666667              400.0]
[               0.0                0.0                1.0]

In [6]:
IM.inverse()

[0.0008571428571428571                   0.0   -0.5142857142857142]
[                  0.0 0.0008571428571428571  -0.34285714285714286]
[                  0.0                   0.0                   1.0]

In [7]:
res = vector(Projector().camera.resolution)
max_point = vector([950, 630])

res = vector(res.list() + [1])
max_point = vector(max_point.list() + [1])

res = IM.inverse() * res
max_point = IM.inverse() * max_point

assert res[2] == max_point[2] == 1
res = res[:2]
max_point = max_point[:2]

max_r = norm(max_point)  # non-distorted
max_r_hat = norm(res)  # distorted
max_point, max_r, res, max_r_hat

((0.30000000000000004, 0.19714285714285706),
 0.35897814156637586,
 (0.5142857142857142, 0.34285714285714286),
 0.6180945043652553)

In [7]:
ineqs = [0 < psi(max_r_hat), psi(max_r_hat) < max_r]
ineqs

[0 < 0.38204081632653053*lambda_1 + 0.14595518533944182*lambda_2 + 1,
 0.38204081632653053*lambda_1 + 0.14595518533944182*lambda_2 + 1 < 0.35897814156637586]

In [8]:
sol = solve_ineq(ineqs)
assert len(sol) == 1
sol[0]

[-2.617521367521368*lambda_1 - 6.851418109430933 < lambda_2,
 lambda_2 < -2.617521367521368*lambda_1 - 4.391908769413204]

In [19]:
psi(max_r_hat).subs(lambda_2 == sol[0][0].lhs())

(5.551115123125783e-17)*lambda_1

In [21]:
psi(max_r_hat).subs(lambda_2 == sol[0][1].rhs())

(5.551115123125783e-17)*lambda_1 + 0.358978141566376

In [9]:
def approx_rationals(expr):
    if expr.operator() is None:  # Reached a leaf node (number or variable)
        return numerical_approx(expr) if expr in QQ else expr
    else:
        return expr.operator()(
            *[approx_rationals(operand) for operand in expr.operands()]
        )


for s in sol[0]:
    display(approx_rationals(s))

-2.61752136752137*lambda_1 - 6.85141810943093 < lambda_2

lambda_2 < -2.61752136752137*lambda_1 - 4.39190876941320

In [10]:
for s in solve(sol[0], lambda_2)[0]:
    display(approx_rationals(s))

-2.61752136752137*lambda_1 - 6.85141810943093 < lambda_2

lambda_2 < -2.61752136752137*lambda_1 - 4.39190876941320

In [11]:
for l1 in [-5, -1, 0, 1, 5]:
    eq = lambda_1 == l1
    display(eq)
    for s in solve(sol[0], lambda_2)[0]:
        display(approx_rationals(s).subs(eq))
    print(
        f"lambda_2 range: {(sol[0][1].subs(eq).rhs() - sol[0][0].subs(eq).lhs()).n()}"
    )
    print()

lambda_1 == -5

6.23618872817591 < lambda_2

lambda_2 < 8.69569806819364

lambda_2 range: 2.45950934001773



lambda_1 == -1

...

lambda_2 range: 2.45950934001773



lambda_1 == 5

-19.9390249470378 < lambda_2

lambda_2 < -17.4795156070200

lambda_2 range: 2.45950934001773



In [12]:
for s in solve(sol[0], lambda_2)[0]:
    print(approx_rationals(s))

-2.61752136752137*lambda_1 - 6.85141810943093 < lambda_2
lambda_2 < -2.61752136752137*lambda_1 - 4.39190876941320
